In [1]:
import pandas as pd
import numpy as np

In [2]:
journal_entries_2020 = pd.read_excel("Vinatics 2020 Journal Entries.xlsx")
trial_balance_2020 = pd.read_excel("Vinatics 2020 Trial Balance.xlsx")

In [3]:
journal_entries_2020.columns

Index(['JE number', 'GL Code', 'Period', 'Effective Date', 'Entry Date',
       'Source', 'Source_Group', 'Functional Amount', 'Preparer ID'],
      dtype='object')

In [4]:
pivot_2020_journals = journal_entries_2020.pivot_table(index = "GL Code", values = "Functional Amount", aggfunc=sum)
pivot_2020_journals.head(5)

,Functional Amount
GL Code,
20000000,4.328652e+06
20000002,-1.634499e+05
20000003,-1.929373e+06
20000004,-6.265112e+05
20000005,-9.460914e+05


In [5]:
pivot_2020_journals = pd.DataFrame(pivot_2020_journals.to_records())
pivot_2020_journals.head(5)

,GL Code,Functional Amount
0,20000000,4.328652e+06
1,20000002,-1.634499e+05
2,20000003,-1.929373e+06
3,20000004,-6.265112e+05
4,20000005,-9.460914e+05


In [6]:
trial_balance_2020.columns

Index(['Account type', 'Account Class', 'GL Account Code', 'GL Account name',
       'Opening Balance', 'Closing Balance'],
      dtype='object')

In [7]:
tb_2020 = trial_balance_2020[['Account type','Account Class', 'GL Account Code', 'GL Account name','Opening Balance','Closing Balance']]
tb_2020 = tb_2020.merge(pivot_2020_journals, how = "left", left_on="GL Account Code", right_on="GL Code").drop("GL Code", axis = 1)
tb_2020.head(5)

,Account type,Account Class,GL Account Code,GL Account name,Opening Balance,Closing Balance,Functional Amount
0,Assets,AR - Trade receivables,20000050,Customer invoices to be issued,3.945206e+05,8.145908e+04,-3.130615e+05
1,Assets,AR - Trade receivables,20000051,Credit notes to be issued,0.000000e+00,0.000000e+00,NaN
2,Assets,AR - Trade receivables,20000000,Receivables - domestic customer,5.439532e+07,5.872397e+07,4.328652e+06
3,Assets,AR - Trade receivables,20000021,Receivables - in receivership,1.038470e+04,8.362068e+03,-2.022636e+03
4,Assets,AR - Trade receivables,20000002,Receivables - customers other sales,-2.898225e+04,-1.924322e+05,-1.634499e+05


In [8]:
tb_2020 = tb_2020.fillna(0)
tb_2020.head(5)

,Account type,Account Class,GL Account Code,GL Account name,Opening Balance,Closing Balance,Functional Amount
0,Assets,AR - Trade receivables,20000050,Customer invoices to be issued,3.945206e+05,8.145908e+04,-3.130615e+05
1,Assets,AR - Trade receivables,20000051,Credit notes to be issued,0.000000e+00,0.000000e+00,0.000000e+00
2,Assets,AR - Trade receivables,20000000,Receivables - domestic customer,5.439532e+07,5.872397e+07,4.328652e+06
3,Assets,AR - Trade receivables,20000021,Receivables - in receivership,1.038470e+04,8.362068e+03,-2.022636e+03
4,Assets,AR - Trade receivables,20000002,Receivables - customers other sales,-2.898225e+04,-1.924322e+05,-1.634499e+05


In [9]:
tb_2020['Calculated Closing Balance'] = tb_2020['Opening Balance'] + tb_2020['Functional Amount']
tb_2020.head(5)

,Account type,Account Class,GL Account Code,GL Account name,Opening Balance,Closing Balance,Functional Amount,Calculated Closing Balance
0,Assets,AR - Trade receivables,20000050,Customer invoices to be issued,3.945206e+05,8.145908e+04,-3.130615e+05,8.145908e+04
1,Assets,AR - Trade receivables,20000051,Credit notes to be issued,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,Assets,AR - Trade receivables,20000000,Receivables - domestic customer,5.439532e+07,5.872397e+07,4.328652e+06,5.872397e+07
3,Assets,AR - Trade receivables,20000021,Receivables - in receivership,1.038470e+04,8.362068e+03,-2.022636e+03,8.362068e+03
4,Assets,AR - Trade receivables,20000002,Receivables - customers other sales,-2.898225e+04,-1.924322e+05,-1.634499e+05,-1.924322e+05


In [11]:
tb_2020['Difference'] = tb_2020['Closing Balance'] - tb_2020['Calculated Closing Balance']
tb_2020_diff = tb_2020.loc[tb_2020['Difference'] > 0]
tb_2020_diff

,Account type,Account Class,GL Account Code,GL Account name,Opening Balance,Closing Balance,Functional Amount,Calculated Closing Balance,Difference
163,Assets,AR - Tax credits,20001101,VAT sales,4.225803e-07,2.818031e-07,-1.445026e-07,2.780778e-07,3.725290e-09
310,Expenses,"Costs for materials, consum., etc",140203004,Packaging purchases,2.613874e+08,4.171158e+08,1.557284e+08,4.171158e+08,5.960464e-08
